In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import openpyxl as pyxl
import pandas as pd
import numpy as np

from bidict import bidict

from excel_helpers import *
from db_entries import *
from unified_excel_sheet import *

import re
from itertools import chain
# we only need the function datetime.datetime.now; we can now reference it as dt.now
from datetime import datetime as dt

import sqlite3 as sql

import logging
logger = logging.getLogger()

In [9]:
%%capture

db_filename = 'my-budget-dev-v4.sqlite'

%run nuclear_option.py $db_filename
%run database_setup.py $db_filename

db = sql.connect(db_filename)

event_groups_df = pd.read_sql('SELECT * FROM event_groups', db, index_col='group_id')

In [10]:
excel_filename = 'GemeinsameBilanzierung_16_17__dev.xlsx'

current_sheet_name = 'August'
comment_sheet_name = 'august_'

# We take here the option data_only since for now we are not interested in the expression
wb = pyxl.load_workbook(excel_filename, data_only=True)


current_sheet = wb.get_sheet_by_name(current_sheet_name)
comment_sheet = wb.copy_worksheet(current_sheet)
comment_sheet.title = comment_sheet_name
comment_sheet.sheet_state = 'hidden'

In [11]:
col_titles_budget = ['budget_pot', 'description', 'date', 'amount', 'excel_range']
col_titles_payments = ['description', 'date', 'amount', 'excel_range']

budgeting = get_df_by_range(comment_sheet, 'A6', 'D130', 2)
budgeting.columns = col_titles_budget

max_bargeld = get_df_by_range(comment_sheet, 'H7', 'J130', 1)
max_bargeld.columns = col_titles_payments
max_bargeld['money_pot'] = 'BM'
max_bargeld['type_description'] = 'Barzahlung'

paul_bargeld = get_df_by_range(comment_sheet, 'K7', 'M130', 1)
paul_bargeld.columns = col_titles_payments
paul_bargeld['money_pot'] = 'BP'
paul_bargeld['type_description'] = 'Barzahlung'

konto = get_df_by_range(comment_sheet, 'N7', 'P130', 1)
konto.columns = col_titles_payments
konto['money_pot'] = 'KG'
konto['type_description'] = 'Kartenzahlung'

all_payments = pd.concat([max_bargeld, paul_bargeld, konto])

all_info = pd.merge(budgeting, all_payments, how='outer', 
                    on=['description', 'date', 'amount'], indicator=True)

all_info['treated'] = 'No'

all_info['date'] = all_info['date'].dt.date
all_info['amount'] = all_info['amount'].round(2)

In [12]:
## Creating Data for Bargeld Payments data

wb_filename = 'full_month.xlsx'
definition_data = 'base_excel.json'

create_excel_table_from_data(all_info, wb_filename, db, definition_data)
checked_data = use_excel_for_data_entry(wb_filename, copy_mode=False)

The file full_month.xlsx will be opened in Excel for data entry. Enter Q to abort, enter anything else to continue: 
Make any input to continue. Abort with Q: 
The read data is:


,ignore,temporary_id,event_type,type_description,description,date,amount,money_pot,money_pot_name,money_pot_sink,...,budget_type,is_recieving,is_payment,is_transfer,in_group,group_name,excel_range_x,excel_range_y,_merge,treated
0,NaN,NaN,K,Kartenzahlung,Miete,2017-08-01,-568.00,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,X,NaN,1.0,Miete,A6:D6,N13:P13,both,No
1,NaN,NaN,K,Kartenzahlung,Miete FFM,2017-08-02,-450.00,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,X,NaN,18.0,Miete FFM,A7:D7,N20:P20,both,No
2,X,NaN,K,Kartenzahlung,Berufsunfähigkeitsversicherung,2017-08-01,-49.05,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,3.0,Berufsunfähigkeitsversicherung,A8:D8,N12:P12,both,No
3,NaN,NaN,K,Kartenzahlung,Strom EnviaM,2017-08-30,-51.00,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,X,NaN,5.0,Strom,A9:D9,N89:P89,both,No
4,NaN,2.0,S,SEPA-Mandat,Vodafone,2017-08-28,-19.99,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,6.0,Vodafone,A10:D10,NaN,left_only,No
5,NaN,2.0,S,SEPA-Mandat,Telefonie,2017-08-28,-0.20,KG,gemeinsames Konto,NaN,...,Miscellaneous,NaN,NaN,NaN,6.0,Vodafone,A11:D11,NaN,left_only,No
6,NaN,NaN,S,SEPA-Mandat,Rechtsschutzversicherung,2017-01-30,-13.90,NaN,NaN,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,4.0,Rechtsschutzversicherung,A12:D12,NaN,left_only,No
7,X,NaN,U,Überweisung,Haftpflichtversicherung,2017-08-01,-7.50,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,2.0,Haftpflichtversicherung,A13:D13,N11:P11,both,No
8,NaN,NaN,S,SEPA-Mandat,GEW,2017-07-05,-2.50,NaN,NaN,NaN,...,Arzeneimittel,NaN,NaN,NaN,16.0,GEW,A14:D14,NaN,left_only,No
9,NaN,NaN,U,Überweisung,Semestergebühr Paul,2017-06-21,-46.15,NaN,NaN,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,12.0,Semestergebühr Paul,A15:D15,NaN,left_only,No



If you are not satisfied with the read data, enter any number. If you do this, the workbook will be opened again. Continue with any other input; abort with Q: 
OK.


In [ ]:
ignored_data_index = checked_data[checked_data['ignore'] == 'X'].index
nonignore_data_index = checked_data.index.difference(ignored_data_index)

ignored_data = checked_data.loc[ignored_data_index]
nonignore_data = checked_data.loc[nonignore_data_index]

nonignore_data = nonignore_data.sort_values('date')
display(nonignore_data)

definition_data = 'base_excel.json'
recheck_wb_filename = 'recheck_full_month.xlsx'

with open(definition_data) as f :
    json_data = f.read()
    sheet_def = json.loads(json_data)

sheet_def['main_sheet']['event_type']['hidden'] = True
sheet_def['main_sheet']['money_pot_name']['hidden'] = True
sheet_def['main_sheet']['money_sink_name']['hidden'] = True
sheet_def['main_sheet']['budget_type']['hidden'] = True

create_excel_table_from_data(nonignore_data, recheck_wb_filename, db, sheet_def)
double_checked = use_excel_for_data_entry(recheck_wb_filename, copy_mode=False)

,ignore,temporary_id,event_type,type_description,description,date,amount,money_pot,money_pot_name,money_pot_sink,...,budget_type,is_recieving,is_payment,is_transfer,in_group,group_name,excel_range_x,excel_range_y,_merge,treated
6,NaN,NaN,S,SEPA-Mandat,Rechtsschutzversicherung,2017-01-30,-13.90,NaN,NaN,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,4.0,Rechtsschutzversicherung,A12:D12,NaN,left_only,No
20,NaN,NaN,K,Kartenzahlung,Backblaze Paul,2017-04-18,-3.77,NaN,NaN,NaN,...,Arzeneimittel,NaN,NaN,NaN,11.0,Backblaze,A26:D26,NaN,left_only,No
16,NaN,NaN,K,Kartenzahlung,Backblaze Max,2017-04-18,-3.77,NaN,NaN,NaN,...,Arzeneimittel,NaN,NaN,NaN,11.0,Backblaze,A22:D22,NaN,left_only,No
9,NaN,NaN,U,Überweisung,Semestergebühr Paul,2017-06-21,-46.15,NaN,NaN,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,12.0,Semestergebühr Paul,A15:D15,NaN,left_only,No
10,NaN,NaN,U,Überweisung,Semestergebühr Max,2017-06-21,-50.56,NaN,NaN,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,13.0,Semestergebühr Max,A16:D16,NaN,left_only,No
11,NaN,NaN,U,Überweisung,GEZ,2017-06-29,-17.50,NaN,NaN,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,17.0,GEZ,A17:D17,NaN,left_only,No
8,NaN,NaN,S,SEPA-Mandat,GEW,2017-07-05,-2.50,NaN,NaN,NaN,...,Arzeneimittel,NaN,NaN,NaN,16.0,GEW,A14:D14,NaN,left_only,No
19,NaN,NaN,K,Kartenzahlung,Monatskarte FFM,2017-07-31,-87.40,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,X,NaN,NaN,NaN,A25:D25,N9:P9,both,No
0,NaN,NaN,K,Kartenzahlung,Miete,2017-08-01,-568.00,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,X,NaN,1.0,Miete,A6:D6,N13:P13,both,No
1,NaN,NaN,K,Kartenzahlung,Miete FFM,2017-08-02,-450.00,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,X,NaN,18.0,Miete FFM,A7:D7,N20:P20,both,No


The file recheck_full_month.xlsx will be opened in Excel for data entry. Enter Q to abort, enter anything else to continue: 
Make any input to continue. Abort with Q: 
The read data is:


,ignore,temporary_id,event_type,type_description,description,date,amount,money_pot,money_pot_name,money_pot_sink,...,budget_type,is_recieving,is_payment,is_transfer,in_group,group_name,excel_range_x,excel_range_y,_merge,treated
6,NaN,NaN,S,SEPA-Mandat,Rechtsschutzversicherung,2017-01-30,-13.90,NaN,NaN,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,4.0,Rechtsschutzversicherung,A12:D12,NaN,left_only,No
20,NaN,NaN,K,Kartenzahlung,Backblaze Paul,2017-04-18,-3.77,NaN,NaN,NaN,...,Arzeneimittel,NaN,NaN,NaN,11.0,Backblaze,A26:D26,NaN,left_only,No
16,NaN,NaN,K,Kartenzahlung,Backblaze Max,2017-04-18,-3.77,NaN,NaN,NaN,...,Arzeneimittel,NaN,NaN,NaN,11.0,Backblaze,A22:D22,NaN,left_only,No
9,NaN,NaN,U,Überweisung,Semestergebühr Paul,2017-06-21,-46.15,NaN,NaN,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,12.0,Semestergebühr Paul,A15:D15,NaN,left_only,No
10,NaN,NaN,U,Überweisung,Semestergebühr Max,2017-06-21,-50.56,NaN,NaN,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,13.0,Semestergebühr Max,A16:D16,NaN,left_only,No
11,NaN,NaN,U,Überweisung,GEZ,2017-06-29,-17.50,NaN,NaN,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,17.0,GEZ,A17:D17,NaN,left_only,No
8,NaN,NaN,S,SEPA-Mandat,GEW,2017-07-05,-2.50,NaN,NaN,NaN,...,Arzeneimittel,NaN,NaN,NaN,16.0,GEW,A14:D14,NaN,left_only,No
19,NaN,NaN,K,Kartenzahlung,Monatskarte FFM,2017-07-31,-87.40,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,X,NaN,NaN,NaN,A25:D25,N9:P9,both,No
0,NaN,NaN,K,Kartenzahlung,Miete,2017-08-01,-568.00,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,X,NaN,1.0,Miete,A6:D6,N13:P13,both,No
1,NaN,NaN,K,Kartenzahlung,Miete FFM,2017-08-02,-450.00,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,X,NaN,18.0,Miete FFM,A7:D7,N20:P20,both,No


In [13]:
parse_unified_excel_table(checked_data, db, db_filename, current_sheet, comment_sheet, excel_filename)

/Users/max/budgeter/unified_excel_sheet.py:267: UserWarning: UNIQUE constraint failed: event_in_group.event_id
  Warning(str(IE))


,ignore,temporary_id,event_type,type_description,description,date,amount,money_pot,money_pot_name,money_pot_sink,...,budget_type,is_recieving,is_payment,is_transfer,in_group,group_name,excel_range_x,excel_range_y,_merge,treated
2,X,NaN,K,Kartenzahlung,Berufsunfähigkeitsversicherung,2017-08-01,-49.05,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,3.0,Berufsunfähigkeitsversicherung,A8:D8,N12:P12,both,No
7,X,NaN,U,Überweisung,Haftpflichtversicherung,2017-08-01,-7.50,KG,gemeinsames Konto,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,NaN,NaN,2.0,Haftpflichtversicherung,A13:D13,N11:P11,both,No
17,X,NaN,K,Kartenzahlung,Sommertickets,2017-07-20,-48.00,NaN,NaN,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,X,NaN,NaN,NaN,A23:D23,NaN,left_only,No
18,X,NaN,K,Kartenzahlung,Sommertickets,2017-08-05,-144.00,NaN,NaN,NaN,...,regelmäßige und budgetierte Ausgaben,NaN,X,NaN,NaN,NaN,A24:D24,NaN,left_only,No
21,X,NaN,K,Kartenzahlung,Fitnessstudio,2017-08-03,-39.80,KG,gemeinsames Konto,NaN,...,Arzeneimittel,NaN,X,NaN,NaN,NaN,A27:D27,N23:P23,both,No
22,X,NaN,K,Kartenzahlung,Aldi,2017-07-31,-23.97,KG,gemeinsames Konto,NaN,...,Lebensmittel,NaN,X,NaN,NaN,NaN,A28:D28,N8:P8,both,No
23,X,NaN,K,Kartenzahlung,Netto,2017-07-31,-9.44,KG,gemeinsames Konto,NaN,...,Lebensmittel,NaN,X,NaN,NaN,NaN,A29:D29,N10:P10,both,No
24,X,NaN,K,Kartenzahlung,Rewe,2017-08-01,-8.71,KG,gemeinsames Konto,NaN,...,Lebensmittel,NaN,X,NaN,NaN,NaN,A30:D30,N14:P14,both,No
25,X,NaN,K,Kartenzahlung,Rossmann,2017-08-01,-9.17,KG,gemeinsames Konto,NaN,...,Driogerieprodukte,NaN,NaN,NaN,NaN,NaN,A31:D31,N15:P15,both,No
26,X,NaN,K,Kartenzahlung,DM,2017-08-01,-9.45,KG,gemeinsames Konto,NaN,...,Driogerieprodukte,NaN,NaN,NaN,NaN,NaN,A32:D32,N16:P16,both,No


In [ ]:
### List of all fully matched things we didnt treat yet
#subsetting_rules = {'_merge' : 'both', 'treated' : 'No'}
#remaining_matched = subset_data_frame(all_info, subsetting_rules)
#
#display(remaining_matched)
#display(untreated_data)

In [ ]:
### For the "GK" (Gemeinsames Konto) Payments we create an excel sheet, where we can change the payment type
#subsetting_rules = {'treated' : 'No'}
#remaining = subset_data_frame(all_info, subsetting_rules)
#remaining = remaining.loc[remaining.index.difference(untreated_data.index)]
#
#wb_filename = 'remaining.xlsx'
#definition_data = 'base_excel.json'
#
#create_excel_table_from_data(remaining, wb_filename, db, definition_data)
#checked_remaining = use_excel_for_data_entry(wb_filename, copy_mode=False)

In [ ]:
#temporary_id_lookup = {}
#
#for index, row in checked_remaining.iterrows() :
#    if row['ignore'] == 'X' :
#        untreated_data = untreated_data.append(checked_full_match_konto.loc[index])
#        continue
#        
#    excel_ranges = ([] if np.isnan(row['excel_range_x']) else [row['excel_range_x']]) + \
#                   ([] if np.isnan(row['excel_range_y']) else [row['excel_range_y']])
#    
#    if not np.isnan(row['temporary_id']) and row['temporary_id'] in temporary_id_lookup :
#        the_id = temporary_id_lookup[row['temporary_id']]
#    else :
#        the_id = create_money_event(db, row['type_description'], row['description'], row['date'])
#        
#
#        full_excel_ranges = [current_sheet.title + '!' + a_range for a_range in excel_ranges]
#    
#        database_comment = autogenerate_database_comment_multiple_excel_ranges(
#            excel_filename, full_excel_ranges, comment_sheet.title)
#        
#        add_database_event(db, the_id, database_comment)
#        
#        if not np.isnan(row['temporary_id']) :
#            temporary_id_lookup[row['temporary_id']] = the_id
#    
#    ## ToDo : Add commenting into database and Excel
#    
#    if row['is_budget_event'] == 'X' :
#        add_budget_event(db, the_id, row['budget_pot'], row['amount'])
#    if row['is_payment'] == 'X' :
#        add_payment(db, the_id, row['money_pot'], row['amount'])
#    if row['is_recieving'] == 'X' :
#        add_payment(db, the_id, row['money_pot'], row['amount'])
#    if row['is_transfer'] == 'X' :
#        add_payment(db, the_id, row['money_pot'], row['money_pot_sink'], row['amount'])
#    
#    if not np.isnan(row['in_group']) :  
#        if row['in_group'] not in event_groups_df.index :
#            if row['group_name'][0] == '=' :
#                raise ValueError(
#                    "You should not use a formula as group name! The given group name was {}.".format(
#                        row['group_name']))
#            crsr = db.cursor()
#            crsr.execute('INSERT INTO event_groups VALUES ({}, "{}")'.format(
#                row['in_group'], row['group_name']))
#            db.commit()
#            event_groups_df = pd.read_sql('SELECT * FROM event_groups', db, index_col='group_id')
#            
#        crsr = db.cursor()
#        crsr.execute('INSERT INTO event_in_group VALUES ({}, {})'.format(row['in_group'], the_id))
#        db.commit()
#    
#    excel_comment = autogenerate_excel_comment(dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
#    excel_cell_list = list(chain.from_iterable([list_from_range_string(a_range) for a_range in ranges]))
#    put_comment_into_excel(comment_sheet, excel_cell_list, excel_comment)
#    
#    all_info.loc[index, 'treated'] = 'Yes'

In [ ]:
wb.save('GemBil.xlsx')

In [ ]:
db.close()

In [14]:
# read a full table into a pandas data framd
crsr = db.cursor()
crsr.execute("SELECT name FROM sqlite_master WHERE type='table';")

for table in crsr.fetchall() :
    display(pd.read_sql_query('SELECT * FROM {};'.format(table[0]), db))

,key,description,liquid
0,KG,gemeinsames Konto,Yes
1,KE,Extrakonto zum gemeinsamen Konto,Yes
2,KM,Konto Max,Yes
3,KP,Konto Paul,Yes
4,KB,Konto Bundesbank,Yes
5,KC,Consorsbankkonto,No
6,BM,Bargeld Max,Yes
7,BP,Bargeld Paul,Yes
8,CB,Chipkarte Bundesbank,Semi
9,CT,Chipkarte Trianon,Semi


,category,type,description,abbreviation
0,Zahlung,Barzahlung,bares Bezahlen,B
1,Zahlung,Kartenzahlung,Zahlen mit Visa oder Girokarte,K
2,Zahlung,Überweisung,Rechnungsbegleichung durch Überweisen,U
3,Zahlung,Dauerauftrag,automatische (regelmäßige) Rechnungsbegleichun...,D
4,Zahlung,SEPA-Mandat,automatische (regelmäßige) Rechnungsbegleichun...,S
5,Zahlung,Bankeinzug,Rechnungsbegleichung durch direkten Bankeinzug,BE
6,Transfer,Abheben,Geldabheben,A
7,Transfer,Aufladen,Aufladen auf eine Chipkarte,AU
8,Transfer,Kontotransfer,Transger von Geld zwischen zwei Konten,KT
9,Transfer,Bargeldtransfer,Transfer von Bargeld zwischen zwei baren Geldt...,BT


,key,description,type
0,L,Lebensmittel,None
1,A,Ausgehen Restaurant,None
2,AE,"Ausgehen Eis, Cafe",A
3,AB,"Ausgehen Döner, Bistro, ...",A
4,AM,"Mensa, Kantine, Kasino",None
5,S,Langlebige Produkte,None
6,SK,Klamotten,S
7,R,regelmäßige und budgetierte Ausgaben,None
8,RM,Mietzahlungen,R
9,RV,Versicherungs- und Vertragsbeträge,R


,id,type,description,date
0,2017013001,SEPA-Mandat,Rechtsschutzversicherung,2017-01-30 00:00:00
1,2017041801,Kartenzahlung,Backblaze Max,2017-04-18 00:00:00
2,2017041802,Kartenzahlung,Backblaze Paul,2017-04-18 00:00:00
3,2017062101,Überweisung,Semestergebühr Paul,2017-06-21 00:00:00
4,2017062102,Überweisung,Semestergebühr Max,2017-06-21 00:00:00
5,2017062901,Überweisung,GEZ,2017-06-29 00:00:00
6,2017070501,SEPA-Mandat,GEW,2017-07-05 00:00:00
7,2017073101,Kartenzahlung,Monatskarte FFM,2017-07-31 00:00:00
8,2017080101,Kartenzahlung,Miete,2017-08-01 00:00:00
9,2017080201,Kartenzahlung,Miete FFM,2017-08-02 00:00:00


,id,money_pot,amount,additional_description,effect_date
0,2017080101,KG,-568.00,None,None
1,2017080201,KG,-450.00,None,None
2,2017083001,KG,-51.00,None,None
3,2017082501,KG,-4.99,None,None
4,2017073101,KG,-87.40,None,None
5,2017081002,KG,-13.99,None,None
6,2017081003,KG,-20.53,None,None
7,2017082801,KG,-20.19,None,None


...

,id,budget_pot,amount,additional_description,budget_effet_date
0,2017080101,R,-568.00,None,None
1,2017080201,R,-450.00,None,None
2,2017083001,R,-51.00,None,None
3,2017082801,R,-19.99,None,None
4,2017082801,M,-0.20,None,None
5,2017013001,R,-13.90,None,None
6,2017070501,G,-2.50,None,None
7,2017062101,R,-46.15,None,None
8,2017062102,R,-50.56,None,None
9,2017062901,R,-17.50,None,None


,group_id,description
0,1,Miete
1,2,Haftpflichtversicherung
2,3,Berufsunfähigkeitsversicherung
3,4,Rechtsschutzversicherung
4,5,Strom
5,6,Vodafone
6,7,Drillisch Paul
7,8,Drillisch Max
8,9,Apple Music Paul
9,10,Spotify Max


,group_id,event_id
0,1,2017080101
1,18,2017080201
2,5,2017083001
3,6,2017082801
4,4,2017013001
5,16,2017070501
6,12,2017062101
7,13,2017062102
8,17,2017062901
9,8,2017081001


,type,description
0,Erstellung,Erstellung eines Eintrages
1,Update,Hinzufügen von Information
2,Korrektur,Korrigieren eines Eintrages
3,Löschung,Löschen eines Eintrages


,id,type,date,description
0,2017080101,Erstellung,2017-11-05,This entry was automatically generated from th...
1,2017080201,Erstellung,2017-11-05,This entry was automatically generated from th...
2,2017083001,Erstellung,2017-11-05,This entry was automatically generated from th...
3,2017082801,Erstellung,2017-11-05,This entry was automatically generated from th...
4,2017013001,Erstellung,2017-11-05,This entry was automatically generated from th...
5,2017070501,Erstellung,2017-11-05,This entry was automatically generated from th...
6,2017062101,Erstellung,2017-11-05,This entry was automatically generated from th...
7,2017062102,Erstellung,2017-11-05,This entry was automatically generated from th...
8,2017062901,Erstellung,2017-11-05,This entry was automatically generated from th...
9,2017081001,Erstellung,2017-11-05,This entry was automatically generated from th...


In [15]:
db_filename = 'my-budget-dev-v4.sqlite'

db = sql.connect(db_filename)

display(pd.read_sql_query('''
    SELECT * FROM 
    (money_events LEFT JOIN budget_events ON money_events.id = budget_events.id)
    LEFT JOIN payments ON money_events.id = payments.id
''', db))

,id,type,description,date,id,budget_pot,amount,additional_description,budget_effet_date,id,money_pot,amount,additional_description,effect_date
0,2017013001,SEPA-Mandat,Rechtsschutzversicherung,2017-01-30 00:00:00,2017013001,R,-13.90,None,None,NaN,None,NaN,None,None
1,2017041801,Kartenzahlung,Backblaze Max,2017-04-18 00:00:00,2017041801,G,-3.77,None,None,NaN,None,NaN,None,None
2,2017041802,Kartenzahlung,Backblaze Paul,2017-04-18 00:00:00,2017041802,G,-3.77,None,None,NaN,None,NaN,None,None
3,2017062101,Überweisung,Semestergebühr Paul,2017-06-21 00:00:00,2017062101,R,-46.15,None,None,NaN,None,NaN,None,None
4,2017062102,Überweisung,Semestergebühr Max,2017-06-21 00:00:00,2017062102,R,-50.56,None,None,NaN,None,NaN,None,None
5,2017062901,Überweisung,GEZ,2017-06-29 00:00:00,2017062901,R,-17.50,None,None,NaN,None,NaN,None,None
6,2017070501,SEPA-Mandat,GEW,2017-07-05 00:00:00,2017070501,G,-2.50,None,None,NaN,None,NaN,None,None
7,2017073101,Kartenzahlung,Monatskarte FFM,2017-07-31 00:00:00,2017073101,R,-87.40,None,None,2.017073e+09,KG,-87.40,None,None
8,2017080101,Kartenzahlung,Miete,2017-08-01 00:00:00,2017080101,R,-568.00,None,None,2.017080e+09,KG,-568.00,None,None
9,2017080201,Kartenzahlung,Miete FFM,2017-08-02 00:00:00,2017080201,R,-450.00,None,None,2.017080e+09,KG,-450.00,None,None


In [16]:
display(pd.read_sql_query('''
    SELECT * FROM 
    (money_events LEFT JOIN budget_events ON money_events.id = budget_events.id)
    LEFT JOIN payments ON money_events.id = payments.id
    WHERE money_events.id = 2017081002
''', db))

,id,type,description,date,id,budget_pot,amount,additional_description,budget_effet_date,id,money_pot,amount,additional_description,effect_date
0,2017081002,SEPA-Mandat,Handy Paul,2017-08-10 00:00:00,2017081002,R,-7.99,None,None,2017081002,KG,-13.99,None,None
1,2017081002,SEPA-Mandat,Handy Paul,2017-08-10 00:00:00,2017081002,M,-6.00,None,None,2017081002,KG,-13.99,None,None
